In [2]:
import zipfile

with zipfile.ZipFile("aligned_data.zip", "r") as z:
    print(z.namelist())

['aligned data/', 'aligned data/val_files', 'aligned data/test_files', 'aligned data/train_files', 'aligned data/c=2/', 'aligned data/c=3/', 'aligned data/c=4/', 'aligned data/c=2/C2E016_2_1.json', 'aligned data/c=2/C2E017_2_1.json', 'aligned data/c=2/C1E027_2_1.json', 'aligned data/c=2/C1E096_2_0.json', 'aligned data/c=2/C1E097_2_0.json', 'aligned data/c=2/C1E004_2_0.json', 'aligned data/c=2/C1E005_2_0.json', 'aligned data/c=2/C2E035_2_0.json', 'aligned data/c=2/C2E034_2_0.json', 'aligned data/c=2/C1E019_2_1.json', 'aligned data/c=2/C1E018_2_1.json', 'aligned data/c=2/C2E028_2_1.json', 'aligned data/c=2/C2E029_2_1.json', 'aligned data/c=2/C1E073_2_0.json', 'aligned data/c=2/C1E072_2_0.json', 'aligned data/c=2/C1E111_2_1.json', 'aligned data/c=2/C1E110_2_1.json', 'aligned data/c=2/C2E042_2_0.json', 'aligned data/c=2/C2E043_2_0.json', 'aligned data/c=2/C1E050_2_1.json', 'aligned data/c=2/C1E106_2_0.json', 'aligned data/c=2/C1E107_2_0.json', 'aligned data/c=2/C1E064_2_1.json', 'aligned d

In [ ]:
import zipfile
import json

train_inputs = []
train_targets = []

with zipfile.ZipFile("aligned_data.zip", "r") as archive:

    files = [f for f in archive.namelist() if f.endswith(".json") and "c=" in f]

    for f in files:
        if len(train_inputs) >= 5000:
            break
        with archive.open(f) as fp:
            try:
                data = json.load(fp)
                for scene in data:
                    chunk = scene.get("CHUNK", "")
                    turns = scene.get("TURNS", [])
                    utterances = [utt for t in turns for utt in t.get("UTTERANCES", [])]

                    if chunk and utterances:
                        input_text = "Summarize: " + " ".join(utterances)
                        train_inputs.append(input_text)
                        train_targets.append(chunk)

                        if len(train_inputs) >= 5000:
                            break
            except Exception as e:
                continue  

print(f"Am extras {len(train_inputs)} exemple valide.")
print(" Exemplu:")
print(train_inputs[1])
print("→", train_targets[1])

✅ Am extras 5000 exemple valide.
📌 Exemplu:
Summarize: Welcome back, everyone. Last we left off The Mighty Nein, after being in proximity to some sort of an assault from Xhorhasian operatives in the city of Zadash here in the center of the Empire, they found themselves also caught up in a murder of the High-Richter in the city. They pulled away and realized there was a manhunt for individuals that were seemingly seen in connection to this murder. As this was transpiring, they had to smuggle out some allies they had found and had been working with, Dolan and Horris. Dolan, part of this whole deal was to get him involved in the local politics. Horris needed to be smuggled out for safety in case they got caught. You made your way to the Evening Nip, met up with Yasha. Down there, you had a conversation with this individual called The Gentleman. You made a deal. You've decided to trust each other on this endeavor and basically do a quid pro quo. A little favor to one, favor for the other a

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from torch.utils.data import DataLoader, Dataset
import torch
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup model
model_id = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Dataset wrapper
class CRD3Dataset(Dataset):
    def __init__(self, inputs, targets, tokenizer, max_input_len=512, max_target_len=128):
        self.inputs = inputs
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_target_len = max_target_len

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_enc = self.tokenizer(self.inputs[idx], truncation=True, padding="max_length", max_length=self.max_input_len, return_tensors="pt")
        target_enc = self.tokenizer(self.targets[idx], truncation=True, padding="max_length", max_length=self.max_target_len, return_tensors="pt")

        input_enc = {k: v.squeeze() for k, v in input_enc.items()}
        target_enc = target_enc["input_ids"].squeeze()
        target_enc[target_enc == tokenizer.pad_token_id] = -100  # Mask loss on padding

        return {**input_enc, "labels": target_enc}

# Dataset & loader
dataset = CRD3Dataset(train_inputs, train_targets, tokenizer)
train_loader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=data_collator)

# Training loop 10 epoci
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 10

model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoca {epoch+1}/{epochs} – Loss mediu: {avg_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


📘 Epoca 1/10 – Loss mediu: 3.0598
📘 Epoca 2/10 – Loss mediu: 2.7982
📘 Epoca 3/10 – Loss mediu: 2.6747
📘 Epoca 4/10 – Loss mediu: 2.5791
📘 Epoca 5/10 – Loss mediu: 2.5118
📘 Epoca 6/10 – Loss mediu: 2.4463
📘 Epoca 7/10 – Loss mediu: 2.3855
📘 Epoca 8/10 – Loss mediu: 2.3387
📘 Epoca 9/10 – Loss mediu: 2.2914
📘 Epoca 10/10 – Loss mediu: 2.2479


In [ ]:
save_path = "finetuned-crd3-flan"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Modelul a fost salvat in folderul: {save_path}")

✅ Modelul a fost salvat în folderul: finetuned-crd3-flan


In [ ]:
import shutil

zip_path = save_path + ".zip"
shutil.make_archive(base_name=save_path, format='zip', root_dir=save_path)

print(f"Arhiva salvata: {zip_path}")

✅ Arhiva salvată: finetuned-crd3-flan.zip


In [ ]:
def generate_prompt_instruction(story):
    if len(story.split()) > 60:
        return (
    f"Extract the most visually striking scene from this story as a prompt for an AI image generator:{story}"
)
    else:
        return (
    "Extract the most visually striking scene from this story as a prompt for an AI image generator . Describe what is physically happening in one sentence." + story
)
import re
def trim_story_by_sentences(story, max_words=150):
    # imparte in propozitii folosind punctuatia de baza
    sentences = re.split(r'(?<=[.!?])\s+', story.strip())

    # from back to front reconstruim
    trimmed = []
    total_words = 0

    for sentence in reversed(sentences):
        word_count = len(sentence.split())
        if total_words + word_count > max_words:
            break
        trimmed.insert(0, sentence.strip())
        total_words += word_count

    return " ".join(trimmed)

# stories = [
#     "You are a brave adventurer, seeking to make a name for yourself in the kingdom. Your journey begins in the small village of River's Crossing, where you hear rumors of a powerful artifact hidden in the nearby mountains. With nothing but your sword, armor, and a few gold coins, you set out on your quest to find the artifact and prove yourself to the kingdom.",
#     "Once upon a time, in a land far away, there was a brave adventurer named John. John had been traveling through the land, seeking adventure and treasure. One day, he stumbled upon an old, abandoned castle on the outskirts of a small village. The castle was said to be cursed, but John was not afraid. He bravely entered the castle, determined to find the treasure that was rumored to be hidden within.As he explored the castle, he encountered many traps and obstacles, but he was able to overcome them all with his quick thinking and bravery. Finally, he reached the treasure room, where he found a chest filled with gold and precious gems. But as he was about to claim his prize, a terrible monster appeared before him, blocking his path.The monster was a fierce dragon, with scales as hard as steel and eyes that glowed with an evil light. John knew that he could not defeat the dragon with his current weapons and armor, so he searched the room for anything that might help him in his battle. He found a sword made of enchanted steel, and a suit of armor that was said to be invincible. With these new weapons and armor, John felt confident that he could defeat the dragon and claim his treasure.As the battle began, John fought fiercely, dodging the dragon's fiery breath and slashing at its scales with his enchanted sword. The dragon was a formidable opponent, but John was determined to win. Finally, after a long and brutal fight, John struck the final blow and the dragon fell to the ground, defeated.With the dragon defeated, John claimed his treasure and left the castle, feeling proud of his accomplishments. He continued his journey, seeking new adventures and treasure, always ready to face whatever challenges lay ahead.",
#     "After leaving the bar, Sam found himself walking down a dark alleyway. He heard footsteps behind him and turned around to see a group of thugs approaching him.",
#     "As you walked through the streets of the city, you couldn't help but feel a sense of unease. The people around you moved with a sense of urgency, and the air was thick with tension. You heard whispers of an impending attack, and you knew you had to do something to help.",
#     "You decide to investigate the rumors yourself and try to find out more information. As you start to ask around, you find that the rumors are true - there is a powerful monster lurking in the nearby forest, and it has been attacking travelers and villagers alike. You decide to gather a group of adventurers to help you take on the monster and put an end to its reign of terror.As you journey through the forest, you encounter various challenges and obstacles. You fight off bandits, navigate treacherous terrain, and even come face to face with some of the monsters that have been terrorizing the area. With your skills and your companions' help, you manage to overcome these challenges and finally reach the monster's lair.The battle is long and hard-fought, but you and your companions emerge victorious. The monster is defeated, and the area is finally safe once again. You and your companions celebrate your victory and decide to continue your adventures together, seeking out new challenges and treasures.",
#     "As you enter the tavern, you can hear the sound of laughter and music coming from within. You make your way to the bar and order a drink, scanning the room for anyone who might know something about the rumors you've heard. After a few minutes, you spot a group of travelers sitting in the corner, map spread out on the table in front of them. You make your way over and introduce yourself, asking if they've heard anything about the monster in the forest. The travelers exchange glances, then one of them speaks up.\\\"Yeah, we've heard the rumors too,\\\" he says. \\\"We were actually planning on heading out to investigate ourselves tomorrow. Would you like to join us?\\\"You agree to join them, and the group spends the rest of the evening planning their strategy and gathering supplies. In the morning, you set out together, determined to put an end to the monster's reign of terror.",
#     "You and your companions have successfully defeated the monster in the forest and have returned to the nearby village to celebrate. The villagers are overjoyed and throw a huge party in your honor. As the night wears on, you start to feel a little restless and decide that you want to continue your adventures. You turn to your companions and suggest that you all go in search of a new quest. Your companions agree, and you all start discussing possible options.",
#     "After discussing with your new companions, you all decide to set out in search of the lost city that is said to be hidden deep within the jungle. You've heard stories of ancient treasures and secrets waiting to be discovered, and you're eager to see if they're true. As you journey deeper into the jungle, you encounter various challenges and obstacles. You battle fierce jungle creatures, navigate treacherous terrain, and even get lost a few times. But with your skills and your companions' help, you manage to overcome these challenges and continue on your quest. Finally, after what feels like weeks of travel, you reach the lost city. It's even more amazing than you could have imagined - towering stone buildings, intricate carvings, and ancient artifacts everywhere you look. You and your companions spend days exploring the city, uncovering its secrets and treasures. In the end, you emerge victorious, with your pockets filled with gold and your hearts filled with adventure. You decide to continue your adventures together, seeking out new challenges and treasures."
# ]

stories = [
    "John had finally reached the village, and he was eager to explore and find some treasure or adventure. He looked around, taking in the sights and sounds of the bustling village. He could hear people talking and laughing, and he could see them going about their daily business. John knew that he had to be careful, though. He didn't want to attract too much attention to himself, as he was still a stranger in this land. John decided to head towards the center of the village, where he hoped he would find some sort of market or bazaar. As he walked, he noticed that the streets were lined with shops and stalls selling all sorts of goods. He saw fruits and vegetables, clothing and jewelry, and even weapons and armor. John knew that he had to be careful not to spend all his gold on frivolous items. He needed to save his money for any treasure or adventure that he might find. As John continued to explore the village, he noticed that there were a lot of people gathered around a particular shop. He decided to investigate and see what all the fuss was about. As he got closer, he saw that the shop was selling rare and exotic items from far-off lands"
]

def clean_prompt_output(text):
    text = text.strip()
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # eliminam duplicari
    unique = []
    for s in sentences:
        if not unique or s.strip() != unique[-1]:
            unique.append(s.strip())
    #  se pastreaza prima prop semnific
    for s in unique:
        if len(s.split()) >= 5:
            return s
    return unique[0] if unique else text


In [ ]:
story = stories[0] 


trimmed_story = trim_story_by_sentences(story, max_words=150)


prompt = generate_prompt_instruction(trimmed_story)


inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=60)


raw_output = tokenizer.decode(output[0], skip_special_tokens=True)
final_output = clean_prompt_output(raw_output)


print("Prompt:\n", prompt)
print("\n Output generat:\n", final_output)

📜 Prompt:
 Extract the most visually striking scene from this story as a prompt for an AI image generator:John decided to head towards the center of the village, where he hoped he would find some sort of market or bazaar. As he walked, he noticed that the streets were lined with shops and stalls selling all sorts of goods. He saw fruits and vegetables, clothing and jewelry, and even weapons and armor. John knew that he had to be careful not to spend all his gold on frivolous items. He needed to save his money for any treasure or adventure that he might find. As John continued to explore the village, he noticed that there were a lot of people gathered around a particular shop. He decided to investigate and see what all the fuss was about. As he got closer, he saw that the shop was selling rare and exotic items from far-off lands

🎨 Output generat:
 As he walked, he noticed that the streets were lining up with shops and stalls, and that there were a lot of people gathered around a partic

In [7]:
!pip install -q evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.3 MB/s eta 0:00:00


In [9]:
!pip install -q rouge_score

  Preparing metadata (setup.py) ... done


In [10]:
from evaluate import load

In [ ]:
import random
from evaluate import load
from jiwer import wer, cer

sample_inputs = train_inputs[:300]
sample_targets = train_targets[:300]

predictions = []
references = []

model.eval()
for input_text in sample_inputs:
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=60)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(pred)
    references.append(input_text)

print(" WER:", wer(references, predictions))
print(" CER:", cer(references, predictions))


bleu = load("bleu")
bleu_score = bleu.compute(predictions=predictions, references=[[r] for r in references])
print(" BLEU:", bleu_score)


rouge = load("rouge")
rouge_score = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
print(" ROUGE-1:", rouge_score["rouge1"])
print(" ROUGE-2:", rouge_score["rouge2"])
print(" ROUGE-L:", rouge_score["rougeL"])

📏 WER: 0.9717607250755287
📏 CER: 0.958574237785703
📊 BLEU: {'bleu': 6.005653039878942e-11, 'precisions': [0.7915153831454232, 0.5523313004131418, 0.42712345428745185, 0.36357934350825843], 'brevity_penalty': 1.1765417133079414e-10, 'length_ratio': 0.041905402537707255, 'translation_length': 10466, 'reference_length': 249753}
📊 ROUGE-1: 0.19352882018108897
📊 ROUGE-2: 0.1318938879904064
📊 ROUGE-L: 0.17384411155341417
